# CS483 - Colab 7
## Decision Trees in Spark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-11-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
!apt update

openjdk-11-jdk-headless is already the newest version (11.0.24+8-1ubuntu3~22.04).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,452 kB]
Get:12 

Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1aJrdYMVmmnUKYhLTlXtyB0FQ9gYJqCrs'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-train.txt')

id='1yLwxRaJIyrC03yxqbTKpedMmHEF86AAq'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-test.txt')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Loading

![MNIST](https://upload.wikimedia.org/wikipedia/commons/thumb/2/27/MnistExamples.png/220px-MnistExamples.png)

In this Colab, we will be using the famous [MNIST database](https://en.wikipedia.org/wiki/MNIST_database), a large collection of handwritten digits that is widely used for training and testing in the field of machine learning.

For your convenience, the dataset has already been converted to the popular LibSVM format, where each digit is represented as a sparse vector of grayscale pixel values.

In [6]:
training = spark.read.format("libsvm").load("mnist-digits-train.txt")
test = spark.read.format("libsvm").load("mnist-digits-test.txt")

# Cache data for multiple uses
training.cache()
test.cache()

DataFrame[label: double, features: vector]

In [7]:
training.show(truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
training.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [9]:
test.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



### Your task

First of all, find out how many instances(samples) we have in our training / test split.

In [10]:
''' 2 lines of code in total expected. '''

print("Number of instances in training set:", training.count())
print("Number of instances in test set:", test.count())

Number of instances in training set: 60000
Number of instances in test set: 10000


Now train a Decision Tree on the training dataset using Spark MLlib.

You can refer to the Python example on this documentation page: [https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier](https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier)

(Note : You **don't** need to and should **NOT** use Indexer as shown by the example, so that your answers can match our solution answers for Gradescope problems.)

Use **seed of 7** (colab 7) for training purposes throughout the colab.

In [11]:
''' 7-8 lines of code in total expected but can differ based on your style. '''
''' For sub-parts of the question (if any), creating different cells of code would be recommended.'''

from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol="features", labelCol="label", seed=7)
model = dt.fit(training)

With the Decision Tree you just induced on the training data, predict the labels of the test set.
Print the predictions for the first 10 digits, and compare them with the labels.

In [12]:
''' 2 lines of code in total expected. '''

predictions = model.transform(test)
predictions.select("prediction", "label").show(10)

+----------+-----+
|prediction|label|
+----------+-----+
|       7.0|  7.0|
|       6.0|  2.0|
|       1.0|  1.0|
|       0.0|  0.0|
|       9.0|  4.0|
|       1.0|  1.0|
|       5.0|  4.0|
|       9.0|  9.0|
|       6.0|  5.0|
|       9.0|  9.0|
+----------+-----+
only showing top 10 rows



The small sample above looks good, but not great!

Let's dig deeper. Compute the accuracy of our model, using the ```MulticlassClassificationEvaluator``` from MLlib.

In [13]:
''' 4-5 lines of code in total expected. '''

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

Accuracy: 0.678


Find out the max depth of the trained Decision Tree, and its total number of nodes.

In [14]:
''' 1 line of code in total expected. '''

print("Max Depth:", model.depth)
print("Number of Nodes:", model.numNodes)

Max Depth: 5
Number of Nodes: 61


It appears that the default settings of the Decision Tree implemented in MLlib did not allow us to train a very powerful model!

Before starting to train a Decision Tree, you can tune the max depth it can reach using the [setMaxDepth()](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.DecisionTreeClassifier.html#pyspark.ml.classification.DecisionTreeClassifier.setMaxDepth) method. Train 21 different DTs, varying the max depth from 0 to 20, endpoints included (i.e., [0, 20]). For each value of the parameter, print the accuracy achieved on the test set, and the number of nodes contained in the given DT.

For this problem, we say that the decision trees overfit the dataset if the classification accuracy starts to drop.

**IMPORTANT:** this parameter sweep can take 30 minutes or more, depending on how busy is your Colab instance. Notice how the induction time grows super-linearly!

In [15]:
''' 6-7 lines of code in total expected. '''
''' For sub-parts of the question (if any), creating different cells of code would be recommended.'''

from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

for depth in range(21):
    dt = DecisionTreeClassifier(featuresCol="features", labelCol="label", seed=7, maxDepth=depth)

    model = dt.fit(training)
    predictions = model.transform(test)
    accuracy = evaluator.evaluate(predictions)

    print("Max Depth:", depth)
    print("Accuracy:", accuracy)
    print("Number of Nodes:", model.numNodes)
    print("-" * 20)

Max Depth: 0
Accuracy: 0.1135
Number of Nodes: 1
--------------------
Max Depth: 1
Accuracy: 0.1994
Number of Nodes: 3
--------------------
Max Depth: 2
Accuracy: 0.3442
Number of Nodes: 7
--------------------
Max Depth: 3
Accuracy: 0.4939
Number of Nodes: 15
--------------------
Max Depth: 4
Accuracy: 0.5942
Number of Nodes: 31
--------------------
Max Depth: 5
Accuracy: 0.678
Number of Nodes: 61
--------------------
Max Depth: 6
Accuracy: 0.7445
Number of Nodes: 121
--------------------
Max Depth: 7
Accuracy: 0.7852
Number of Nodes: 245
--------------------
Max Depth: 8
Accuracy: 0.8194
Number of Nodes: 489
--------------------
Max Depth: 9
Accuracy: 0.8492
Number of Nodes: 929
--------------------
Max Depth: 10
Accuracy: 0.8664
Number of Nodes: 1677
--------------------
Max Depth: 11
Accuracy: 0.8721
Number of Nodes: 2683
--------------------
Max Depth: 12
Accuracy: 0.8768
Number of Nodes: 3749
--------------------
Max Depth: 13
Accuracy: 0.8791
Number of Nodes: 4683
---------------

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-629c54ed190d>", line 12, in <cell line: 9>
    model = dt.fit(training)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/ml/base.py", line 205, in fit
    return self._fit(dataset)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/ml/wrapper.py", line 381, in _fit
    java_model = self._fit_java(dataset)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/ml/wrapper.py", line 378, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.10/dist-packages/pyspark/errors/exceptions/captured.py", line 179, in deco
    return f(*a, **kw)
  File "/usr/loca

ConnectionRefusedError: [Errno 111] Connection refused

Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!